# Lab 2 Extra: Multiple Output Gaussian Processes
### Gaussian Process Summer School 2018
_written by Wil Ward, adapted from notebooks by Ricardo Andrade Pacheco and [Neil Lawrence](http://inverseprobability.com/)_

This lab is introduces Gaussian process approximation of vector-valued functions, i.e. functions with multiple outputs. You should make sure you have completed Lab 2 before attempting.

Extra labs are for you to explore in your own time, giving details of other uses of Gaussian processes not covered in the summer school. Answers for extra labs will be made available _after_ the summer school.

In [ ]:
# Support for maths
import numpy as np
# Plotting tools
from matplotlib import pyplot as plt
# we use the following for plotting figures in jupyter
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

# GPy: Gaussian processes library
import GPy

## 1. A Dataset with Multiple Outputs
Do show the capabilities of Gaussian processes for modelling multi-output functions, we will use the result times of the winner of the Olympic sprint finals from 1986 to 2008. Sprinting events consist of 100 m, 200 m and 400 m races, and there are men's and women's events, so in total the dimension of our output space is $6$. 

We have provided the data to download from the [Labs page of the summer school](http://gpss.cc/gpss18/labs). You should download this into the same directory as this notebook. Execute the following cell to read in the data.

Data Source: _Rogers, S. and Girolami, M., 2016. A First Course in Machine Learning. CRC Press. ISBN:978-1498738484_

In [ ]:
# We load the olympic sprints dataset (Rogers and Girolami, 2016)
data = np.genfromtxt('olympic_sprints.csv', delimiter=',')

# Assign our inputs (year) and observations (winning time in seconds)
X, y = data[:,0][:, None], data[:,1][:, None]
# We also have label information to indicate the type of race 
labels = data[:,2][:, None]

For now, we'll plot the results. We can deal with the show the result by label, and observe a few features of the data. For example, we can observe correlation between the race times, and consistent trends between race types, and between men and women's events.

In [ ]:
markers = ['bo', 'b^', 'bs', 'ro', 'r^', 'rs']
events = ["Women's 100 m", "Women's 200 m", "Women's 400 m", "Men's 100 m", "Men's 200 m", "Men's 400 m"]

plt.figure(figsize=(14, 8))
for i in range(6):
    x_event = X[np.nonzero(labels == i)]
    y_event = y[np.nonzero(labels == i)]
    plt.plot(x_event, y_event, markers[i])

# Annotate plots
plt.title('Olympic Sprint Times')
plt.xlabel('year'), plt.ylabel('time [s]')
plt.legend(labels=events);

## 2. Fitting Gaussian Processes to Individual Events

Even though we are using a shared space to model these events, we can model each independently. We first try modelling each individual race event with its own GP:

In [ ]:
Xnew = np.arange(1884,2021)[:, None]

plt.figure(figsize=(14, 8))
# Plot measurements first
for i in range(6):
    x_event = X[np.nonzero(labels == i)]
    y_event = y[np.nonzero(labels == i)]
    plt.plot(x_event, y_event, markers[i])
# Fit a GP to each event and plot the fit
for i in range(6):
    k = GPy.kern.RBF(1, lengthscale=80) # RBF kernel
    x_event = X[np.nonzero(labels == i)][:, None] # Get the years that we have measurements for
    y_event = y[np.nonzero(labels == i)][:, None] # Get the corresponding race time
    m = GPy.models.GPRegression(x_event, y_event, k) # Fit a basic GP
    m.optimize() # Optimise the hyperparameters
    # Predict the times from 1884 to 
    mean, Cov = m.predict(Xnew, full_cov=True)
    # We plot the event measurements and mean GP fit with 95% CI
    plt.plot(Xnew, mean, "b-" if i < 3 else "r-", x_event, y_event, markers[i])
    plt.fill_between(Xnew[:,0], mean[:,0] - 1.96*np.sqrt(np.diag(Cov)), mean[:,0] + 1.96*np.sqrt(np.diag(Cov)), color=markers[i][0], alpha=0.2)

# Annotate plots
plt.title('Individual GP regression fits to Olympic sprint times')
plt.xlabel('year'), plt.ylabel('time [s]')
plt.legend(labels=events);

As we can see, the individual regression fits give reasonable predictions and our data lies within the confidence interval. However, ascan be seen particularly in the Women's 200 m and 400 m, we are missing a lot of data for the earlier years. We can exploit the capabilities of a _coregionalised_ Gaussian process to use the results we have from the Men's races on these data to make better inferences where we have missing data for the Women's races.

## 3. Coregionalisation
In `GPy`, we deal with multiple output data in a particular way. We need to specify the index of the output we are interested in for modelling as an _input_ to our model. In our individual regression fits, we only used the _year_ as our input and sepearated the fits by their race type. In the corregionalised model, we will augment our inputs with the event label and use both as input to get the race output.

In [ ]:
X = data[:,(0,2)] # We augment our input with the event type
print(X.shape)

We will look at modelling the data using a coregionalisation approach. This involves using the Kronecker product of kernels, which we can perform in `GPY` using the (overloaded) `**` operator. Typically in Python, this operator is the exponent operator, but for `GPy` kernels, we use it to indicate a tensor product. Linear models of coregionalisation are based on combining a matrix with a standard covariance function, or kernel. We think of this matrix as a paritcular _type_ of covariance function, whose elements are referenced using the indices denoting the particular events (e.g. $0$ is "_Women's 100 m_", etc.) Using this setup, we want to build a covariance matrix where the first column of the input features (in this case, the event year) is passed to a covariance function for regression, and the second column (the event labels) are passed to the coregionalisation matrix.

We will start by using the intrinsic coregionalisation model, also known as _multitask Gaussian processes_. These are handily given as a kernel in GPy: `GPy.kern.Coregionalize`.

In [ ]:
k = GPy.kern.RBF(1, lengthscale=80) ** GPy.kern.Coregionalize(1, output_dim = 6, rank = 5)

The coregionalisation matrix, $\mathbf{B}$, is in itself constructed from two _other_ matrices, $\mathbf{B} = \mathbf{W}\mathbf{W}^\text{T} + \mathrm{diag}(\kappa)$. This allows us to specify a low rank form for our coregionalisation matrix. However, in the above example, we have specified that it is not constrained to have a low rank form: `rank = 5`. Here we are specifying the rank of $\mathbf{W}\mathbf{W}^\text{T}$, which when combined with the diagonal matrix from $\kappa$ gives us a general coregionalisation matrix, $\mathbf{B}$.

We can now use our combined Kronecker product matrix (that is the $\mathbf{K} \otimes \mathbf{B}$) with a standard GP regression model, which, in `GPy`, we can do as usual:

In [ ]:
m = GPy.models.GPRegression(X, y, k)
m.optimize()
# Preview the model
m

We can plot the results using our ability to reference the desired output event as an input. When we predict from the GP, we can simply make the value of the second column of our prediction locations equal to the index of the event.

We can observe now in the plot below the results of the coregionalised model:

In [ ]:
Xnew = np.arange(1880,2021)[:, None]

plt.figure(figsize=(14, 8))
# Plot measurements first
for i in range(6):
    x_event = X[np.nonzero(labels == i)]
    y_event = y[np.nonzero(labels == i)]
    plt.plot(x_event, y_event, markers[i])
    
# Predict an event using our coregionslied GP
for i in range(6):
    # We predict at input (Xnew, i) where i is the index of the event
    mean, Cov = m.predict(np.hstack((Xnew, i + np.zeros(Xnew.shape))), full_cov=True)
    # We plot the event measurements and mean GP fit with 95% CI
    plt.plot(Xnew, mean, "b-" if i < 3 else "r-", x_event, y_event, markers[i])
    plt.fill_between(Xnew[:,0], mean[:,0] - 1.96*np.sqrt(np.diag(Cov)), mean[:,0] + 1.96*np.sqrt(np.diag(Cov)), color=markers[i][0], alpha=0.2)

# Annotate plots
plt.title('Coregionalised GP regression fits to Olympic sprint times')
plt.xlabel('year'), plt.ylabel('time [s]')
plt.legend(labels=events);

We have introduced here a simple assumption of covariance across the coregionalised space, but we can still see the clear benefits of the coregionalisation when compared with the individual fits. This is clear, paricularly in the reasonable predictions of the Women's 200 m where we have little data. We can observe the benefits of using joint asusmptions, but there are many more things we can with the model. For example, each of the races are of different lengths, so the series will have different means. We can add another coregionalisation term to deal with the mean. We will use a `Bias` kernel to try and model the different mean times. We will try this, along with using a coregionalisation model of rank $1$:

In [ ]:
k1 = GPy.kern.RBF(1, lengthscale=80) ** GPy.kern.Coregionalize(1, output_dim=6, rank=1)
k2 = GPy.kern.Bias(1) ** GPy.kern.Coregionalize(1, output_dim=6, rank=1)
k  = k1 + k2

In [ ]:
m = GPy.models.GPRegression(X, y, k)
m.optimize()
# Preview model
m

In [ ]:
Xnew = np.arange(1880,2021)[:, None]

plt.figure(figsize=(14, 8))
# Plot measurements first
for i in range(6):
    x_event = X[np.nonzero(labels == i)]
    y_event = y[np.nonzero(labels == i)]
    plt.plot(x_event, y_event, markers[i])
    
# Predict an event using our coregionslied GP
for i in range(6):
    # We predict at input (Xnew, i) where i is the index of the event
    mean, Cov = m.predict(np.hstack((Xnew, i + np.zeros(Xnew.shape))), full_cov=True)
    # We plot the event measurements and mean GP fit with 95% CI
    plt.plot(Xnew, mean, "b-" if i < 3 else "r-", x_event, y_event, markers[i])
    plt.fill_between(Xnew[:,0], mean[:,0] - 1.96*np.sqrt(np.diag(Cov)), mean[:,0] + 1.96*np.sqrt(np.diag(Cov)), color=markers[i][0], alpha=0.2)

# Annotate plots
plt.title('Coregionalised GP regression with RBF + Bias terms fit to Olympic sprint times')
plt.xlabel('year'), plt.ylabel('time [s]')
plt.legend(labels=events);

So far we have used a simple linear model of coregionalisation. We can observe that it is particularly confident, even in the absence of data, especially in the Women's 400 m event before 1960. We might think that the model is perhaps being overconfident, where we have forced too much information sharing between the sprints. We can return to an intrinsic coregionalisation model, and force the two base covariance functions to share the same coregionalisation matrix:

In [ ]:
k1 = GPy.kern.RBF(1, lengthscale=80)  + GPy.kern.Bias(1) 
k  = k1 ** GPy.kern.Coregionalize(1, output_dim=6, rank=5)

In [ ]:
m = GPy.models.GPRegression(X, y, k)
m.optimize()
# Preview model
m

In [ ]:
Xnew = np.arange(1880,2021)[:, None]

plt.figure(figsize=(14, 8))
# Plot measurements first
for i in range(6):
    x_event = X[np.nonzero(labels == i)]
    y_event = y[np.nonzero(labels == i)]
    plt.plot(x_event, y_event, markers[i])
    
# Predict an event using our coregionslied GP
for i in range(6):
    # We predict at input (Xnew, i) where i is the index of the event
    mean, Cov = m.predict(np.hstack((Xnew, i + np.zeros(Xnew.shape))), full_cov=True)
    # We plot the event measurements and mean GP fit with 95% CI
    plt.plot(Xnew, mean, "b-" if i < 3 else "r-", x_event, y_event, markers[i])
    plt.fill_between(Xnew[:,0], mean[:,0] - 1.96*np.sqrt(np.diag(Cov)), mean[:,0] + 1.96*np.sqrt(np.diag(Cov)), color=markers[i][0], alpha=0.2)

# Annotate plots
plt.title('Coregionalised GP regression with RBF + Bias terms and intrinsic coregionalisation fit to Olympic sprint times')
plt.xlabel('year'), plt.ylabel('time [s]')
plt.legend(labels=events);

### Exercise

Can you fix the issues of over confidence in the model? You might try including, for example,
 - an additional covariance function to handle short lengthscale effects
 - changing the rank of the coregionalisation matrix(/ces)
 - Adding a coregionalised noise model using the white noise kernel, `GPy.kern.White`